<a href="https://colab.research.google.com/github/Andy-Lewis-Sapner/ITCC-Ant-SemB24/blob/master/HW2/ITCC_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions#

- To start the application, please run all of the cells (from start to finish). Running all of them is possible by choosing Runtime -> Run all (or CTRL + F9).
- In order to use the application, please go to the Main Program section and open it. It is encouraged to use a full screen of the output, and is possible by choosing the 3 dots on the cell, and then "View output fullscreen".
- The login credentials for this application, in this stage of development are "testuser" (username) and "testpassword" (password).
- After completing the login process, the home screen is shown. You may choose any one of the four screens, and look at them.

#Installations#

In [104]:
!pip install squarify firebase requests beautifulsoup4

#Imports#

In [105]:
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout, interact, HBox, VBox
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import squarify
import textwrap
import random
import re
import networkx as nx
from collections import Counter
import json
from firebase import firebase
import requests
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer

from google.colab import drive, files

#Utils#

##DB Functionality##

In [106]:
def connect_to_DB(DB_url):
 return firebase.FirebaseApplication(DB_url, None)

In [107]:
def saveJsonToDB(jsonFile):
  FBConn.post('/onShapeJSON/', jsonFile)

In [108]:
def openJsonFromDB():
  result = FBConn.get('/onShapeJSON/', None)
  return list(result.values())[-1]

##Global Variables##

In [109]:
filters_dict = {'Graph': 'temp', 'Project Name': 'temp', 'Student Name': None, 'Tab Name': None, 'Start Date': 'date1', 'End Date': 'date2', 'Actions Type': False}
DB_url = 'https://onshapeassistant-default-rtdb.firebaseio.com/'
glossary_url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
index_is_posted = False
json_is_posted = False
menu_buttons_list = []
data = openJsonFromDB()

##Index Creation##

In [110]:
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

In [111]:
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

In [112]:
def get_stop_words():
  words = [
    "a", "able", "about", "above", "abst", "accordance", "according", "accordingly", "across", "act", "actually",
    "added", "adj", "affected", "affecting", "affects", "after", "afterwards", "again", "against", "ah", "all",
    "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and",
    "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways",
    "anywhere", "apparently", "approximately", "are", "aren", "arent", "arise", "around", "as", "aside", "ask",
    "asking", "at", "auth", "available", "away", "awfully", "b", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind",
    "being", "believe", "below", "beside", "besides", "between", "beyond", "biol", "both", "brief", "briefly", "but",
    "by", "c", "ca", "came", "can", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com",
    "come", "comes", "contain", "containing", "contains", "could", "couldnt", "d", "date", "did", "didn't",
    "different", "do", "does", "doesn't", "doing", "done", "don't", "down", "downwards", "due", "during", "e",
    "each", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough",
    "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere",
    "ex", "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows",
    "for", "former", "formerly", "forth", "found", "four", "from", "further", "furthermore", "g", "gave", "get",
    "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "had",
    "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "hed", "hence", "her", "here",
    "hereafter", "hereby", "herein", "heres", "hereupon", "hers", "herself", "hes", "hi", "hid", "him", "himself",
    "his", "hither", "home", "how", "howbeit", "however", "hundred", "i", "id", "ie", "if", "i'll", "im", "immediate",
    "immediately", "importance", "important", "in", "inc", "indeed", "index", "information", "instead", "into",
    "invention", "inward", "is", "isn't", "it", "itd", "it'll", "its", "itself", "i've", "j", "just", "k", "keep",
    "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter",
    "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "ll", "look",
    "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means",
    "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "more", "moreover", "most", "mostly",
    "mr", "mrs", "much", "mug", "must", "my", "myself", "n", "na", "name", "namely", "nay", "nd", "near", "nearly",
    "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety",
    "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing",
    "now", "nowhere", "o", "obtain", "obtained", "obviously", "of", "off", "often", "oh", "ok", "okay", "old",
    "omitted", "on", "once", "one", "ones", "only", "onto", "or", "ord", "other", "others", "otherwise", "ought",
    "our", "ours", "ourselves", "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages", "part",
    "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible",
    "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly",
    "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "re", "readily",
    "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively",
    "research", "respectively", "resulted", "resulting", "results", "right", "run", "s", "said", "same", "saw",
    "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen",
    "self", "selves", "sent", "seven", "several", "shall", "she", "shed", "she'll", "shes", "should", "shouldn't",
    "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since",
    "six", "slightly", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime",
    "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying",
    "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup",
    "sure", "t", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that",
    "that'll", "thats", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there",
    "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'll", "theyre", "they've", "think", "this", "those",
    "thou", "though", "thoughh", "thousand", "throug", "through", "throughout", "thru", "thus", "til", "tip", "to",
    "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two",
    "u", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "ups", "us",
    "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "ve",
    "very", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "was", "wasnt", "way", "we", "wed", "welcome",
    "we'll", "went", "were", "werent", "we've", "what", "whatever", "what'll", "whats", "when", "whence", "whenever",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which",
    "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "whose",
    "why", "widely", "willing", "wish", "with", "within", "without", "wont", "words", "world", "would", "wouldnt",
    "www", "x", "y", "yes", "yet", "you", "youd", "you'll", "your", "youre", "yours", "yourself", "yourselves",
    "you've", "z", "zero"
  ]
  return words

In [113]:
def remove_stop_words(index):
    stop_words = get_stop_words()
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

In [114]:
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [115]:
def find_important_10_words(url):
  soup = fetch_page(url)
  if soup is not None:
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    important10 = dict(sorted(index.items(), key=lambda item: item[1]))
    important10 = list(important10.keys())[-11:]
    return important10, index
  return None

In [116]:
def create_important_10_index(important10, initial_index):
  index = dict()
  for word in important10:
    index[word] = initial_index[word]
  return index

In [117]:
def post_index_to_DB():
  important10, initial_index = find_important_10_words(glossary_url)
  index = create_important_10_index(important10, initial_index)
  FBConn.post('/onshapeIndex/', index)
  index_is_posted = True

In [118]:
def get_index():
  return list(FBConn.get('/onshapeIndex/', None).values())[-1]

#All Screens#

##Login Screen##

In [119]:
login_dict = {'Username': 'temp', 'Password': 'temp'}
# output = widgets.Output()

In [120]:
def create_login_screen():
  grid = GridspecLayout(100, 100, height='400px')
  grid.add_class("login-screen")


  def on_button_clicked(b):
      login_dict['Username'] = grid[20:27, 38:58].value #= dropdown_graphtype()
      login_dict['Password'] = grid[30:37, 38:58].value #= project_name()
      if login_dict['Username'] == 'testuser' and login_dict['Password'] == 'testpassword':
        page.children = [create_home_screen()]

  def create_expanded_button(description, button_style):
      button = Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
      return button

  # Place the widgets with height of 7 rows each
  grid[10:20, 48:78] = widgets.Label(value='Login', layout=Layout(color='black'))
  grid[20:27, 38:58] = login_username()
  grid[30:37, 38:58] = login_password()
  grid[43:53, 46:53] = create_expanded_button('Login', 'success')

  grid[43:53, 46:53].on_click(on_button_clicked)
  login_page = widgets.VBox([grid], layout=widgets.Layout(margin='300px auto'))
  return login_page

def login_username():
  return widgets.Text(
    placeholder='Enter Username',
    description='Username:',
    disabled=False
)

def login_password():
  return widgets.Password(
    placeholder='Enter password',
    description='Password:',
    disabled=False
  )

##Home Screen##

In [121]:
def json_upload_clicked(b):
  page.children = [create_menu(), create_json_upload_screen()]

In [122]:
def index_search_clicked(b):
  page.children = [create_menu(), create_index_search_screen()]

In [123]:
def parameters_clicked(b):
  page.children = [create_menu(), create_parameters_screen()]

In [124]:
def statistics_clicked(b):
  page.children = [create_menu(), create_statistics_screen()]

In [125]:
def create_home_screen():
  welcome_label = widgets.Label("Welcome to OnShape Project Manager Assist")
  welcome_label.add_class("welcome-label")

  json_upload_screen = widgets.Button(description="JSON Upload")
  json_upload_screen.add_class("ju")
  json_upload_screen.on_click(json_upload_clicked)

  index_search_screen = widgets.Button(description="Index Search", layout=widgets.Layout(disabled=True))
  index_search_screen.add_class("is")
  index_search_screen.on_click(index_search_clicked)

  parameters_screen = widgets.Button(description="Parameters", layout=widgets.Layout(disabled=True))
  parameters_screen.add_class("ps")
  parameters_screen.on_click(parameters_clicked)

  statistics_screen = widgets.Button(description="Statistics", layout=widgets.Layout(disabled=True))
  statistics_screen.add_class("sts")
  statistics_screen.on_click(statistics_clicked)

  menu_buttons_list.extend([json_upload_screen, index_search_screen, parameters_screen, statistics_screen])

  hbox1 = widgets.HBox([parameters_screen, statistics_screen])
  hbox2 = widgets.HBox([json_upload_screen, index_search_screen])

  buttons_vbox = widgets.VBox([hbox1, hbox2])
  buttons_vbox.add_class("buttons-vbox")
  home_screen = widgets.VBox([welcome_label, buttons_vbox])
  home_screen.add_class("home-screen")
  return home_screen

##Parameters Screen##

###Parameters Menu Screen###

In [126]:
def create_parameters_screen():
  # output = widgets.Output()
  grid = GridspecLayout(100, 100, height='400px')
  grid.add_class('param-menu')


  def on_button_clicked(b):
      filters_dict['Graph'] = grid[20:27, 20:50].label #= dropdown_graphtype()
      filters_dict['Project Name'] = grid[27:34, 20:50].label #= project_name()
      filters_dict['Studnet Name'] = grid[34:41, 20:50].label #= studet_name()
      filters_dict['Tab Name'] = grid[41:48, 20:50].label #= card_name()
      filters_dict['Start Date'] = grid[48:55, 20:50].value #= start_date_picker()
      filters_dict['End Date'] = grid[48:55, 20:50].value #=  end_date_picker()
      filters_dict['Actions Type'] = grid[55:62, 20:50].label #=  realType()
      page.children = [create_menu(), create_parameters_results_screen(filters_dict, data)]

  def create_expanded_button(description, button_style):
    button = Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
    return button

  # Place the widgets with height of 7 rows each
  grid[10:20, 40:60] = widgets.Label(value='Filter Zone')
  grid[20:27, 20:50] = dropdown_graphtype()
  grid[27:34, 20:50] = project_name()
  grid[34:41, 20:50] = studet_name()
  grid[34:41, 50:] = widgets.Label(value='(optional)')
  grid[41:48, 20:50] = tab_name()
  grid[41:48, 50:] = widgets.Label(value='(optional)')
  grid[48:55, 20:50] = start_date_picker()
  grid[48:55, 50:] = end_date_picker()
  grid[55:62, 20:50] = realType()
  grid[90:, 45:55] = create_expanded_button('Next', 'success')

  grid[90:, 45:55].on_click(on_button_clicked)
  return grid


def dropdown_graphtype():
  return widgets.Dropdown(
    options=[('Bar Plot', 1), ('Histogram', 2), ('Pie Chart', 3), ('Treemap', 4)],
    value=2,
    description='Type:',
    layout=Layout(height='auto', width='auto'),
  )

def project_name():
  return widgets.Dropdown(
    options=[('P1', 1), ('P2', 2), ('P3', 3), ('P4', 4)],
    value=2,
    description='Project:',
    layout=Layout(height='auto', width='auto'),
  )

def studet_name():
  return widgets.Dropdown(
    options=[('S1', 1), ('S2', 2), ('S3', 3), ('S4', 4)],
    value=2,
    description='Student:',
    layout=Layout(height='auto', width='auto'),
  )

def tab_name():
  return widgets.Dropdown(
    options=[('S1', 1), ('S2', 2), ('S3', 3), ('S4', 4)],
    value=2,
    description='Tab:',
    layout=Layout(height='auto', width='auto'),
  )
def start_date_picker():
  return widgets.DatePicker(
    description='Start Date:',
    disabled=False
  )
def end_date_picker():
  return widgets.DatePicker(
    description='End Date:',
    disabled=False
  )

def realType():
  return widgets.Dropdown(
    options=[('All Data', 1), ('Real Data', 2)],
    value=1,
    description='Data:',
    layout=Layout(height='auto', width='auto'),
  )

###Parameters Results Screen###

In [127]:
def activities_quantity(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S")

    # Convert the filter dates to datetime format
    start_date = pd.to_datetime(filters_dict['Start Date'])
    end_date = pd.to_datetime(filters_dict['End Date'])

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Filter the DataFrame based on the date range
    df = df[(df['Time'] >= start_date) & (df['Time'] <= end_date)]

    # Check if the date range spans multiple months
    months_range = end_date.month != start_date.month

    # Create the 'activity_time' column based on the date range
    if months_range:
        df['activity_time'] = df['Time'].dt.strftime('%m/%Y')
    else:
        df['activity_time'] = df['Time'].dt.strftime('%d/%m')

    # Group by 'User' and 'activity_time' and count the activities
    activity_counts = df.groupby(['User', 'activity_time']).size().unstack(fill_value=0)

    if activity_counts.empty:
        plt.plot()
        return plt

    # Plot the data
    activity_counts.T.plot(kind='bar', stacked=True)

    plt.xlabel('Date')
    plt.xticks(rotation=90)
    plt.ylabel('Number of activities')
    plt.title('Activities per User in Project "' + filters_dict['Project Name'] + '"')
    plt.legend(title='User')
    plt.tight_layout()

    return plt

In [128]:
def work_hours(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S")

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Filter by 'Student Name' if provided
    if filters_dict['Student Name'] is not None:
        df = df[df['User'] == filters_dict['Student Name']]

    # Get unique users
    users = df['User'].unique()

    points_of_time = []
    for user in users:
        user_df = df[df['User'] == user]

        if user_df.empty:
            continue

        user_work_hours = user_df['Time'].dt.strftime("%H:%M")
        min_time = min(user_work_hours)
        max_time = max(user_work_hours)
        start_time = datetime.strptime(min_time, "%H:%M")
        end_time = datetime.strptime(max_time, "%H:%M")
        time_diff = (end_time - start_time).total_seconds() / 3600  # time_diff in hours
        points_of_time.append([user, start_time, time_diff])

    fig, ax = plt.subplots()
    for user, start_time, time_diff in points_of_time:
        ax.barh(user, time_diff, left=start_time.hour + start_time.minute / 60, label=user)

    # Formatting the x-axis to display time properly
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:02}:{:02}'.format(int(x), int((x % 1) * 60))))

    plt.xlabel('Time of Day')
    plt.ylabel('User')
    plt.title('Work Hours per User in Project "' + filters_dict['Project Name'] + '"')
    plt.legend()
    plt.tight_layout()
    return plt

In [129]:
def relative_work(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Filter the DataFrame based on the 'Tab Name' and 'Project Name'
    df = df[(df['Tab'] == filters_dict['Tab Name']) & (df['Document'] == filters_dict['Project Name'])]

    # Check if 'Student Name' filter is applied
    if filters_dict['Student Name'] is not None:
        df['User'] = df['User'].apply(lambda x: 'Others' if x != filters_dict['Student Name'] else x)

    # Count the occurrences of each user
    quantities = df['User'].value_counts()

    # Plot the data as a pie chart
    plt.pie(quantities.values, labels=quantities.index, autopct='%1.1f%%')
    plt.axis('equal')
    plt.title('Relative Work on Tab "' + filters_dict['Tab Name'] + '" in Project "' + filters_dict['Project Name'] + '"')
    plt.tight_layout()

    return plt

In [130]:
def shorten_description(description):
    # Define patterns to shorten
    patterns = [
        (r'\b(Tab [A-Z][a-z]* [0-9]+)', r'\1'),
        (r'\b(Stop|Start) assembly drag', r'\1 drag'),
        (r'\b(Tab [A-Z][a-z]*) \.\.\.', r'\1 is'),
        (r'\b(Tab [A-Z][a-z]*) of type [A-Z]+ (closed|opened) by [A-Za-z]+', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) of type [A-Z]+ (closed|opened)', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) (closed|opened) by [A-Za-z]+', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) (closed|opened)', r'\1 \2'),
        (r'Tab "(.*?)" of type BLOB', r'\1'),
    ]

    # Apply patterns to shorten the description
    for pattern, replacement in patterns:
        description = re.sub(pattern, replacement, description)

    return description

def common_activities_in_project(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Count occurrences of each activity description
    activities = df['Description'].value_counts()

    # Calculate average occurrence
    average = activities.mean()

    # Filter activities with counts >= 2 * average
    activities = activities[activities >= 2 * average]

    # Shorten activity descriptions
    activities.index = activities.index.map(shorten_description)

    # Wrap long labels
    wrapped_labels = [textwrap.fill(label, 10) for label in activities.index]

    # Prepare data for plotting
    activities = dict(zip(wrapped_labels, activities.values))

    # Generate bright colors for plotting
    colors = [(random.uniform(0.4, 1), random.uniform(0.4, 1), random.uniform(0.4, 1)) for _ in range(len(activities))]

    # Create plot using squarify
    fig, ax = plt.subplots()
    squarify.plot(sizes=activities.values(), label=activities.keys(), color=colors, ax=ax, alpha=0.7)

    # Adjust text labels size
    for label in ax.texts:
        label.set_fontsize(8)

    plt.title('Activities in Project "' + filters_dict['Project Name'] + '"')
    plt.axis('off')
    plt.tight_layout()

    return plt

In [131]:
def filters_data():
  filters_hboxes = [widgets.HBox([title := widgets.Label(value='Chosen Filters', layout=widgets.Layout(margin='0 auto', font_size='16px'))])]
  title.add_class('title')
  for key, value in filters_dict.items():
    if key is None or value is None:
      continue
    if key == 'Actions Type':
      value = 'Positive' if value else 'All'
    hbox = widgets.HBox([widgets.Label(value=key), widgets.Label(value=str(value), layout=widgets.Layout(margin="0 auto"))])
    filters_hboxes.append(hbox)

  filters = widgets.VBox(filters_hboxes, layout=widgets.Layout(margin="150px auto", height="75%"))
  filters.add_class('filters')
  filters.add_class('component')
  return filters


def create_graph(filters_dict, data):
  graph_output = widgets.Output()
  plt = None
  error = widgets.Label('Graph cannot be created')
  error.add_class('error')
  with graph_output:
    match filters_dict['Graph']:
      case 'AQ':
        plt = activities_quantity(filters_dict, data)
      case 'WH':
        plt = work_hours(filters_dict, data)
      case 'RW':
        plt = relative_work(filters_dict, data)
      case 'CA':
        plt = common_activities_in_project(filters_dict, data)
    if plt is not None:
      plt.show()
    else:
      display(error)

  graph = widgets.VBox([graph_output], layout=widgets.Layout(margin="90px auto", height="75%"))
  graph.add_class('component')
  return graph


def create_parameters_results_screen(filters_dict, data):
  screen_title = widgets.Label(value="Parameters Screen", layout=widgets.Layout(height="100%"))
  screen_title.add_class('screen-title')
  screen_hbox = widgets.HBox([filters_data(), create_graph(filters_dict, data)])
  screen = widgets.VBox([screen_title, screen_hbox])
  screen.add_class('screen')
  display(HTML(style))
  return screen

style = """
<style>
    .screen .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .screen .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 16px auto;
      width: 70%;
      padding: 4px;
    }
    .screen {
      # background: linear-gradient(65deg, #e8d49e, #ebddb7);
      max-width: 100%;
    }
    .component {
      border: 2px dotted #261e0c;
      padding: 4px;
    }
    .filters {
      width: 40%;
    }
    .screen .widget-button {
      margin: 16px auto;
      background-color: #14b363;
    }
    .screen .widget-button:hover {
      background-color: #0d7842;
    }
    .screen .widget-button:active {
      background-color: #77b300;
    }
    .screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
</style>
"""

##Statistics Screen##

In [132]:
# Load the data
df = pd.DataFrame(data)

# Convert 'Time' to datetime and sort by time
df['Time'] = pd.to_datetime(df['Time'])
df = df.sort_values('Time')

# Get the minimum and maximum dates from the data
min_date = df['Time'].min().date()
max_date = df['Time'].max().date()

# Function to filter data based on user selections
def filter_data(df, start_date, end_date, user):
    if start_date:
        start_date = pd.to_datetime(start_date)
        df = df[df['Time'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        df = df[df['Time'] <= end_date]
    if user != 'All':
        df = df[df['User'] == user]
    return df

# Function to add value labels to a bar graph
def add_value_labels(ax, spacing=5):
    for rect in ax.patches:
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2
        label = f"{y_value:.2f}"
        va = 'bottom' if y_value >= 0 else 'top'
        ax.annotate(label, (x_value, y_value), xytext=(0, spacing), textcoords="offset points", ha='center', va=va)

# Function to export data to Excel
def export_to_excel(data, filename):
    data.to_excel(filename, index=False)
    files.download(filename)

# Function to show negative actions
def show_negative_actions(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the number of negative actions (deletions, closures, cancellations) performed by each user.")
        negative_actions = filtered_df[filtered_df['Description'].str.contains('Delete|Close|Cancel', case=False)]
        negative_count = negative_actions['User'].value_counts()

        fig, ax = plt.subplots(figsize=(10, 6))
        negative_count.plot(kind='bar', ax=ax)
        add_value_labels(ax)

        plt.title('Number of Negative Actions by User')
        plt.xlabel('User')
        plt.ylabel('Number of Negative Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(negative_count.reset_index(name='Negative Actions'), '/content/negative_actions_data.xlsx'))

# Function to show project statistics
def show_project_stats(filtered_df, days=7):
    output.clear_output(wait=True)
    with output:
        print("This section provides an overview of the project statistics, including the duration, total actions, number of unique tabs, and number of unique users.")

        project_duration = filtered_df['Time'].max() - filtered_df['Time'].min()
        total_actions = len(filtered_df)
        unique_tabs = filtered_df['Tab'].nunique()
        unique_users = filtered_df['User'].nunique()

        # Average actions per day
        days_active = (filtered_df['Time'].max() - filtered_df['Time'].min()).days + 1
        avg_actions_per_day = total_actions / days_active

        # Percentage of negative actions
        negative_actions = filtered_df[filtered_df['Description'].str.contains('Delete|Close|Cancel', case=False)]
        percent_negative_actions = (len(negative_actions) / total_actions) * 100

        # Percentage of active users
        active_users = filtered_df['User'].nunique()
        total_users = df['User'].nunique()
        percent_active_users = (active_users / total_users) * 100

        # Most active day
        filtered_df.loc[:, 'Date'] = filtered_df['Time'].dt.date
        actions_per_day = filtered_df.groupby('Date').size()
        most_active_day = actions_per_day.idxmax()
        most_active_day_count = actions_per_day.max()

        # Least active day
        least_active_day = actions_per_day.idxmin()
        least_active_day_count = actions_per_day.min()

        # Peak hour of activity
        filtered_df.loc[:, 'Hour'] = filtered_df['Time'].dt.hour
        actions_per_hour = filtered_df.groupby('Hour').size()
        peak_hour = actions_per_hour.idxmax()
        peak_hour_count = actions_per_hour.max()

        # Unique documents
        unique_documents = filtered_df['Document'].nunique()

        recent_date = filtered_df['Time'].max() - timedelta(days=days)
        active_docs = filtered_df[filtered_df['Time'] > recent_date]['Document'].unique()
        inactive_docs = set(filtered_df['Document'].unique()) - set(active_docs)

        stats_html = f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center; color: black;">
            <h2 style="color: #4A4A4A;">Project Statistics</h2>
            <p><strong>Project Duration:</strong> {project_duration}</p>
            <p><strong>Total Actions:</strong> {total_actions}</p>
            <p><strong>Number of Unique Tabs:</strong> {unique_tabs}</p>
            <p><strong>Number of Unique Users:</strong> {unique_users}</p>
            <p><strong>Average Actions Per Day:</strong> {avg_actions_per_day:.2f}</p>
            <p><strong>Percentage of Negative Actions:</strong> {percent_negative_actions:.2f}%</p>
            <p><strong>Percentage of Active Users:</strong> {percent_active_users:.2f}%</p>
            <p><strong>Most Active Day:</strong> {most_active_day} ({most_active_day_count} actions)</p>
            <p><strong>Least Active Day:</strong> {least_active_day} ({least_active_day_count} actions)</p>
            <p><strong>Peak Hour of Activity:</strong> {peak_hour}:00 ({peak_hour_count} actions)</p>
            <p><strong>Unique Documents:</strong> {unique_documents}</p>
            <p><strong>Active Documents (Last {days} Days):</strong> {', '.join(active_docs)}</p>
            <p><strong>Inactive Documents:</strong> {', '.join(inactive_docs)}</p>
        </div>
        """
        display(widgets.HTML(value=stats_html))

    project_stats_data = {
        'Metric': [
            'Project Duration',
            'Total Actions',
            'Number of Unique Tabs',
            'Number of Unique Users',
            'Average Actions Per Day',
            'Percentage of Negative Actions',
            'Percentage of Active Users',
            'Most Active Day',
            'Most Active Day Count',
            'Least Active Day',
            'Least Active Day Count',
            'Peak Hour of Activity',
            'Peak Hour Count',
            'Unique Documents',
            'Active Documents (Last 7 Days)',
            'Inactive Documents'
        ],
        'Value': [
            project_duration,
            total_actions,
            unique_tabs,
            unique_users,
            avg_actions_per_day,
            percent_negative_actions,
            percent_active_users,
            most_active_day,
            most_active_day_count,
            least_active_day,
            least_active_day_count,
            peak_hour,
            peak_hour_count,
            unique_documents,
            ', '.join(active_docs),
            ', '.join(inactive_docs)
        ]
    }
    export_button.on_click(lambda b: export_to_excel(pd.DataFrame(project_stats_data), '/content/project_stats_data.xlsx'))

# Function to show user progress
def show_user_progress(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph shows the cumulative number of actions performed by each user over time, indicating their activity trends.")
        filtered_df['Date'] = filtered_df['Time'].dt.date
        user_actions = filtered_df.groupby(['User', 'Date']).size().unstack(fill_value=0).cumsum(axis=1)
        plt.figure(figsize=(12, 6))
        for user in user_actions.index:
            plt.plot(user_actions.columns, user_actions.loc[user], label=user)
        plt.title('Cumulative Actions by User Over Time')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Actions')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(user_actions.reset_index(), '/content/user_progress_data.xlsx'))

# Function to show user effectiveness
def show_user_effectiveness(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the average number of daily actions for each user, providing insights into their activity levels.")
        daily_actions = filtered_df.groupby(['User', filtered_df['Time'].dt.date]).size().reset_index(name='Actions')
        avg_daily_actions = daily_actions.groupby('User')['Actions'].mean().sort_values(ascending=False)

        fig, ax = plt.subplots(figsize=(10, 6))
        avg_daily_actions.plot(kind='bar', ax=ax)
        add_value_labels(ax)

        plt.title('Average Daily Actions by User')
        plt.xlabel('User')
        plt.ylabel('Average Daily Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(avg_daily_actions.reset_index(name='Average Daily Actions'), '/content/user_effectiveness_data.xlsx'))

# Function to show action types
def show_action_types(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the most common types of actions performed in the project, highlighting the primary activities.")
        action_types = filtered_df['Description'].value_counts()
        plt.figure(figsize=(12, 6))
        action_types.head(10).plot(kind='bar')
        plt.title('Top 10 Most Common Action Types')
        plt.xlabel('Action Type')
        plt.ylabel('Count')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

        top_actions_html = f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center;">
            <h2 style="color: #4A4A4A;">Top 10 Most Common Action Types</h2>
            <ul style="display: inline-block; text-align: left;">
                {''.join([f'<li>{action}</li>' for action in action_types.head(10).index])}
            </ul>
        </div>
        """
        display(widgets.HTML(value=top_actions_html))

    export_button.on_click(lambda b: export_to_excel(action_types.head(10).reset_index(name='Count'), '/content/action_types_data.xlsx'))

# Function to show activity by day of week
def activity_by_day_of_week(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph shows the distribution of activity across different days of the week, providing insights into weekly activity patterns.")
        filtered_df['Day'] = filtered_df['Time'].dt.day_name()
        day_counts = filtered_df['Day'].value_counts().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
        plt.figure(figsize=(10, 6))
        day_counts.plot(kind='bar')
        plt.title('Activity by Day of Week')
        plt.xlabel('Day of Week')
        plt.ylabel('Number of Actions')
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(day_counts.reset_index(name='Count'), '/content/activity_by_day_of_week_data.xlsx'))

# Function to show action type ratio
def action_type_ratio(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This pie chart shows the ratio of different types of actions (Create, Edit, Delete, Add) in the project.")
        action_types = filtered_df['Description'].apply(lambda x: 'Create' if 'Create' in x else ('Edit' if 'Edit' in x else ('Delete' if 'Delete' in x else 'Add')))
        type_counts = action_types.value_counts()
        type_counts = type_counts[type_counts.index.isin(['Create', 'Edit', 'Delete', 'Add', 'Other'])]
        plt.figure(figsize=(8, 8))
        plt.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
        plt.title('Ratio of Action Types')
        plt.axis('equal')
        plt.show()

    export_button.on_click(lambda b: export_to_excel(type_counts.reset_index(name='Count'), '/content/action_type_ratio_data.xlsx'))

# Function to show user collaboration
def user_collaboration(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This network graph shows how users are connected through their work on shared documents, indicating collaboration levels.")
        user_doc = filtered_df.groupby('Document')['User'].unique().apply(list)
        G = nx.Graph()
        for users in user_doc:
            for u1 in users:
                for u2 in users:
                    if u1 != u2:
                        if G.has_edge(u1, u2):
                            G[u1][u2]['weight'] += 1
                        else:
                            G.add_edge(u1, u2, weight=1)
        pos = nx.spring_layout(G)
        plt.figure(figsize=(12, 8))
        nx.draw(G, pos, with_labels=True, node_color='lightblue',
                node_size=500, font_size=8, font_weight='bold')
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        plt.title('User Collaboration Network')
        plt.axis('off')
        plt.show()

    export_button.on_click(lambda b: export_to_excel(filtered_df[['User', 'Document']].drop_duplicates(), '/content/user_collaboration_data.xlsx'))

# Function to calculate effective work time
def calculate_effective_work_time(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis calculates the effective work time for each user, considering breaks longer than 30 minutes as non-work time.")
        effective_times = {}
        for user in filtered_df['User'].unique():
            user_actions = filtered_df[filtered_df['User'] == user].sort_values('Time')
            total_time = 0
            for i in range(1, len(user_actions)):
                time_diff = (user_actions.iloc[i]['Time'] - user_actions.iloc[i-1]['Time']).total_seconds() / 60
                if time_diff <= 30:  # if time difference is less than 30 minutes, consider it work time
                    total_time += time_diff
            effective_times[user] = total_time / 60  # convert to hours

        plt.figure(figsize=(10, 6))
        plt.bar(effective_times.keys(), effective_times.values())
        plt.title('Effective Work Time per User')
        plt.xlabel('User')
        plt.ylabel('Effective Work Time (Hours)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(pd.DataFrame(list(effective_times.items()), columns=['User', 'Effective Work Time']), '/content/effective_work_time_data.xlsx'))

# Function to analyze trends over time
def analyze_trends_over_time(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis shows how the number of actions changes over the course of the project, indicating activity trends.")
        filtered_df['Date'] = filtered_df['Time'].dt.date
        daily_actions = filtered_df.groupby('Date').size().reset_index(name='Actions')

        plt.figure(figsize=(12, 6))
        plt.plot(daily_actions['Date'], daily_actions['Actions'])
        plt.title('Number of Actions Over Time')
        plt.xlabel('Date')
        plt.ylabel('Number of Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(daily_actions, '/content/trends_over_time_data.xlsx'))

# Function to analyze work styles
def analyze_work_styles(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis categorizes users based on their work patterns, providing insights into their work styles. 'Late Night' represents actions performed between 22:00 and 05:00, 'Short Bursts' represents actions performed within 5 minutes of the previous action, 'Weekend' represents actions performed on weekends, and 'Regular' represents actions not falling into the above categories.")
        user_patterns = {}
        for user in filtered_df['User'].unique():
            user_actions = filtered_df[filtered_df['User'] == user]
            total_actions = user_actions.shape[0]

            # Late night work (22:00-05:00)
            late_night = user_actions[user_actions['Time'].dt.hour.isin([22, 23, 0, 1, 2, 3, 4])].shape[0]

            # Short work bursts (less than 5 minutes between actions)
            short_bursts = sum(user_actions['Time'].diff().dt.total_seconds() < 300)

            # Weekend work
            weekend_work = user_actions[user_actions['Time'].dt.dayofweek.isin([5, 6])].shape[0]

            user_patterns[user] = {
                'Late Night': late_night / total_actions,
                'Short Bursts': short_bursts / total_actions,
                'Weekend': weekend_work / total_actions,
                'Regular': max(0, 1 - (late_night + short_bursts + weekend_work) / total_actions)  # remaining time
            }

        patterns_df = pd.DataFrame(user_patterns).T

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

        # Bar graph
        patterns_df.plot(kind='bar', stacked=True, ax=ax1)
        ax1.set_title('Work Styles of Users (Proportions)')
        ax1.set_xlabel('User')
        ax1.set_ylabel('Proportion of Actions')
        ax1.legend(title='Work Style', bbox_to_anchor=(1.05, 1), loc='upper left')

        # Table with exact data
        cell_text = []
        for user, data in patterns_df.iterrows():
            cell_text.append([f"{value:.2%}" for value in data])

        ax2.axis('tight')
        ax2.axis('off')
        table = ax2.table(cellText=cell_text, rowLabels=patterns_df.index, colLabels=patterns_df.columns,
                          cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        ax2.set_title('Work Styles of Users (Exact Percentages)')

        plt.tight_layout()
        plt.show()

        explanation_html = """
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center; color: black;">
            <h2 style="color: #4A4A4A;">Explanation of Work Styles</h2>
            <p><strong>Late Night:</strong> Proportion of actions performed between 22:00 and 05:00</p>
            <p><strong>Short Bursts:</strong> Proportion of actions performed within 5 minutes of the previous action</p>
            <p><strong>Weekend:</strong> Proportion of actions performed on weekends</p>
            <p><strong>Regular:</strong> Proportion of actions not falling into the above categories</p>
        </div>
        """
        display(widgets.HTML(value=explanation_html))

    export_button.on_click(lambda b: export_to_excel(patterns_df.reset_index(), '/content/work_styles_data.xlsx'))

# Function to update statistics based on selected filters
def update_statistics(statistic, start_date, end_date, user):
    filtered_df = filter_data(df, start_date, end_date, user)
    if statistic == 'Negative Actions':
        show_negative_actions(filtered_df)
    elif statistic == 'Project Stats':
        show_project_stats(filtered_df)
    elif statistic == 'User Progress':
        show_user_progress(filtered_df)
    elif statistic == 'User Effectiveness':
        show_user_effectiveness(filtered_df)
    elif statistic == 'Action Types':
        show_action_types(filtered_df)
    elif statistic == 'Activity by Day of Week':
        activity_by_day_of_week(filtered_df)
    elif statistic == 'Action Type Ratio':
        action_type_ratio(filtered_df)
    elif statistic == 'User Collaboration':
        user_collaboration(filtered_df)
    elif statistic == 'Effective Work Time':
        calculate_effective_work_time(filtered_df)
    elif statistic == 'Trends Over Time':
        analyze_trends_over_time(filtered_df)
    elif statistic == 'Work Styles':
        analyze_work_styles(filtered_df)

# UI Design
title_html = """
<div style="background-color: #99b3ff; color: white; padding: 20px; text-align: center;">
    <h1 style="font-family: Arial, sans-serif;">Data Analysis System</h1>
</div>
"""

# Widget for selecting statistics
statistic_widget = widgets.Dropdown(
    options=['Negative Actions', 'Project Stats',
             'User Progress', 'User Effectiveness', 'Action Types',
             'Activity by Day of Week', 'Action Type Ratio',
             'User Collaboration',
             'Effective Work Time', 'Trends Over Time', 'Work Styles'],
    value='Negative Actions',
    description='Select Statistic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='20%', margin='0 10px 0 0')
)

# Widget for selecting start date
start_date_widget = widgets.DatePicker(
    description='Start Date:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='20%', margin='0 10px 0 0'),
    value=min_date
)

# Widget for selecting end date
end_date_widget = widgets.DatePicker(
    description='End Date:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='20%', margin='0 10px 0 0'),
    value=max_date
)

# Widget for selecting user
user_widget = widgets.Dropdown(
    options=['All'] + list(df['User'].unique()),
    value='All',
    description='Select User:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='20%', margin='0 10px 0 0')
)

# Button for loading the selected filters
load_button = widgets.Button(
    description='Load Selection',
    button_style='success',
    layout=widgets.Layout(width='20%', margin='10px 0 0 0')
)

# Button for exporting data
export_button = widgets.Button(
    description='Export Data',
    button_style='info',
    layout=widgets.Layout(width='20%', margin='10px 0 0 0')
)

# Output widget to display graphs
output = widgets.Output()
output.add_class('statistics-output')

# Function to handle button click
def on_button_click(b):
    update_statistics(statistic_widget.value, start_date_widget.value, end_date_widget.value, user_widget.value)

load_button.on_click(on_button_click)

# Styling the widgets
statistic_widget.style.description_width = 'initial'
start_date_widget.style.description_width = 'initial'
end_date_widget.style.description_width = 'initial'
user_widget.style.description_width = 'initial'
load_button.style.button_color = '#4CAF50'
export_button.style.button_color = '#2196F3'

# Function to create the screen
def create_statistics_screen():
  statistics_screen = widgets.VBox([widgets.HTML(value=title_html), HBox([statistic_widget, start_date_widget, end_date_widget, user_widget]), HBox([load_button, export_button]), output], layout=widgets.Layout(height="850px"))
  statistics_screen.add_class('statistics-screen')
  return statistics_screen

##Index Search Screen##

In [133]:
def create_index_search_screen():
      # Create the headline
    screen_title = widgets.Label(
        value="Search by Index",
        layout=widgets.Layout(height="auto", width="100%")
    )
    screen_title.add_class('screen-title')

    # Create search input
    search_input = widgets.Text(
        value='',
        placeholder='Enter search term',
        description='Search:',
        disabled=False
    )
    search_input.add_class('centered-widget')

    # Create search button
    search_button = widgets.Button(
        description='Search',
        disabled=False,
        button_style='',
        tooltip='Click to search',
        icon='search'
    )

    # Create output area for search results
    search_output = widgets.HTML(
        value='<p>Search results will appear here</p>',
        layout=widgets.Layout(
            border='1px solid black',
            padding='10px',
            margin='10px 0px',
            max_height='300px',
            overflow_y='auto',
            width='80%'
        )
    )
    search_output.add_class('output')

    # Sample data for demonstration
    sample_data = {
        "apple": {
            "description": "A round fruit with red or green skin and a white interior.",
            "nutrition": "Rich in fiber and vitamin C.",
            "uses": "Eaten raw, baked in pies, or made into juice."
        },
        "banana": {
            "description": "A long curved fruit with a yellow skin when ripe.",
            "nutrition": "Good source of potassium and vitamin B6.",
            "uses": "Eaten raw, used in smoothies, or baked in bread."
        },
        "orange": {
            "description": "A round citrus fruit with a tough bright reddish-yellow rind.",
            "nutrition": "High in vitamin C and antioxidants.",
            "uses": "Eaten fresh, juiced, or used in marmalades."
        },
        "grape": {
            "description": "A small, sweet fruit growing in clusters on a vine.",
            "nutrition": "Contains resveratrol and other antioxidants.",
            "uses": "Eaten fresh, dried as raisins, or made into wine."
        },
        "pear": {
            "description": "A sweet, juicy fruit with a rounded body tapering towards the stalk.",
            "nutrition": "Good source of fiber and vitamin C.",
            "uses": "Eaten fresh, poached, or used in baking."
        }
    }

    def on_search_clicked(b):
        search_term = search_input.value.lower()
        results = []
        for fruit, info in sample_data.items():
            if search_term in fruit or any(search_term in value.lower() for value in info.values()):
                results.append((fruit, info))

        if results:
            output = "<h3>Search Results:</h3>"
            for fruit, info in results:
                output += f"""
                <div style="margin-bottom: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; color: black;">
                    <h4 style="color: black;">{fruit.capitalize()}</h4>
                    <p><strong>Description:</strong> {info['description']}</p>
                    <p><strong>Nutrition:</strong> {info['nutrition']}</p>
                    <p><strong>Uses:</strong> {info['uses']}</p>
                </div>
                """
        else:
            output = f"<p>No results found for '{search_term}'.</p>"

        search_output.value = output

    search_button.on_click(on_search_clicked)

    # Create a horizontal box layout for the search input and button
    search_hbox = widgets.HBox(
        [search_input, search_button],
        layout=widgets.Layout(justify_content='center', align_items='center', margin='0 auto')
    )

    # Create a vertical box layout for the search input/button and output
    content_vbox = widgets.VBox(
        [search_hbox, search_output],
        layout=widgets.Layout(align_items='center', justify_content='center', width='80%')
    )

    # Create the main screen layout with a top-aligned headline
    screen = widgets.VBox(
        [screen_title, content_vbox],
        layout=widgets.Layout(
            align_items='center',
            justify_content='flex-start',
            width='100%',
            height='100vh',
            padding='10px',
            margin='0 auto'
        )
    )
    screen.add_class('screen')

    display(HTML(style))
    return screen

style = """
<style>
    .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 20px auto; /* Ensure space below the title */
      width: 100%;
      padding: 10px;
      color: black;
    }
    .screen {
      background-image: url("https://gradients.mijo-design.com/public/uploads/files/g9.png");
      background-size: cover; /* Ensures the image covers the area */
      background-position: center; /* Centers the background image */
      max-width: 100%;
      display: flex;
      flex-direction: column;
      justify-content: flex-start; /* Ensure the title is at the top */
      align-items: center;
      height: 100vh; /* Full viewport height */
      margin: 0 auto;
    }
    .component {
      border: 2px dotted #261e0c;
      padding: 4px;
    }
    .filters {
      width: 40%;
    }
    .widget-button {
      margin: 16px auto;
      background-color: green;
      border-radius: 5px;
    }
    .widget-button:hover {
      background-color: darkgreen;
    }
    .widget-button:active {
      background-color: #77b300;
    }
    .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
    .centered-widget {
      display: flex;
      justify-content: center;
      align-items: center;
      flex-direction: column;
    }
    .output {
      color: black; /* Set the text color of the output to black */
    }
</style>
"""

##JSON Upload Screen##

In [134]:
def enableMenu():
  json_is_posted = True
  for button in menu_buttons_list:
    button.layout.disabled = False
  data = openJsonFromDB()

In [135]:
def create_json_upload_screen():
    screen_title = widgets.Label(value="Manager Screen", layout=widgets.Layout(height="auto", width="100%"))
    screen_title.add_class('screen-title')

    file_upload = widgets.FileUpload(
        accept='.json',
        multiple=False,
        description='Upload JSON',
        button_style=''
    )
    file_upload.add_class('centered-widget')

    upload_output = widgets.Output()

    header_area = widgets.Textarea(
        value='',
        description='Header:',
        disabled=False,
        layout=widgets.Layout(width='120%', height='60px', font_size='30px')
    )
    header_area.add_class('custom-header-textarea')


    paragraph_area = widgets.Textarea(
        value='',
        description='Text:',
        disabled=False,
        layout=widgets.Layout(width='120%', height='60px', font_size='30px')
    )
    paragraph_area.add_class('custom-paragraph-textarea')

    json_display_area = widgets.Textarea(
        value='',
        description='JSON Content:',
        disabled=True,
        layout=widgets.Layout(width='50%', height='300px', font_size='12px')
    )
    json_display_area.add_class('component')

    next_button = widgets.Button(
        description='Next',
        button_style='',
        layout=widgets.Layout(width='50px', height='50px', font_size='20px', margin='10px')
    )

    back_button = widgets.Button(
        description='Back',
        button_style='',
        layout=widgets.Layout(width='50px', height='50px', font_size='20px', margin='10px')
    )

    button_box = widgets.HBox([back_button, next_button], layout=widgets.Layout(justify_content='center', width='70%'))

    content_display = widgets.VBox([header_area, paragraph_area, button_box], layout=widgets.Layout(align_items='center'))

    current_index = [0]  # Using a list to store mutable state
    headers_and_paragraphs = []

    def on_next_click(b):
        if current_index[0] < len(headers_and_paragraphs) - 1:
            current_index[0] += 1
            header, paragraph = headers_and_paragraphs[current_index[0]]
            header_area.value = header
            paragraph_area.value = paragraph
        else:
            header_area.value = "End of headers"
            paragraph_area.value = "End of paragraphs"

    def on_back_click(b):
        if current_index[0] > 0:
            current_index[0] -= 1
            header, paragraph = headers_and_paragraphs[current_index[0]]
            header_area.value = header
            paragraph_area.value = paragraph
        else:
            header_area.value = "Start of headers"
            paragraph_area.value = "Start of paragraphs"

    next_button.on_click(on_next_click)
    back_button.on_click(on_back_click)

    def on_file_upload(change):
        nonlocal headers_and_paragraphs
        with upload_output:
            upload_output.clear_output()
            for fname, file_info in file_upload.value.items():
                content = file_info['content']
                print(f'Uploaded file name: {fname}')

                try:
                    json_data = json.loads(content.decode("utf-8"))

                    if not isinstance(json_data, list):
                        json_data = [json_data]  # Convert to list if it's not already

                    saveJsonToDB(json_data)
                    enableMenu()

                    headers_and_paragraphs = []
                    for item in json_data:
                        if isinstance(item, dict):
                            for key, value in item.items():
                                headers_and_paragraphs.append((key, str(value)))
                        elif isinstance(item, str):
                            # Assume the entire string is a paragraph without a header
                            headers_and_paragraphs.append(("Paragraph", item))
                        else:
                            headers_and_paragraphs.append(("Unknown", str(item)))

                    current_index[0] = 0  # Reset to first header and paragraph
                    if headers_and_paragraphs:
                        header, paragraph = headers_and_paragraphs[0]
                        header_area.value = header
                        paragraph_area.value = paragraph
                    else:
                        header_area.value = "No headers found"
                        paragraph_area.value = "No paragraphs found in the JSON file"

                    # Update JSON display area
                    json_display_area.value = json.dumps(json_data, indent=4)

                except json.JSONDecodeError:
                    header_area.value = 'Error: Invalid JSON file'
                    paragraph_area.value = ''
                    json_display_area.value = ''

    file_upload.observe(on_file_upload, names='value')

    screen_hbox = widgets.HBox(
        [file_upload, upload_output],
        layout=widgets.Layout(justify_content='center', align_items='center')
    )

    main_hbox = widgets.HBox(
        [content_display, json_display_area],
        layout=widgets.Layout(justify_content='center', align_items='flex-start', width='100%')
    )

    manager_screen = widgets.VBox(
        [screen_title, screen_hbox, main_hbox],
        layout=widgets.Layout(align_items='center', justify_content='center', width='100%')
    )
    manager_screen.add_class('screen')

    display(HTML(style))
    return manager_screen

style = """
<style>
    .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 16px auto;
      width: 70%;
      padding: 4px;
    }
    .screen {
      background-image: url("https://gradients.mijo-design.com/public/uploads/files/g9.png");
      background-size: cover; /* Ensures the image covers the area */
      background-position: center; /* Centers the background image */
      max-width: 100%;
      display: flex;
      justify-content: center;
      align-items: center;
      height: full; /* Full viewport height */
    }
    .component {
      border: 2px dotted #261e0c;
      padding: 4px;

    }
    .filters {
      width: 40%;
    }
    .widget-button {
      margin: 16px auto;
      background-color: green;
      border-radius: 5px;
    }
    .widget-button:hover {
      background-color: darkgreen;
    }
    .widget-button:active {
      background-color: #77b300;
    }
    .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
    .centered-widget {
      display: flex;
      justify-content: center;
      align-items: center;
      flex-direction: column;
    }
    .custom-header-textarea .widget-label {
      color: black;
    }
    .custom-paragraph-textarea .widget-label {
      color: black;
    }
</style>
"""

#Main Page#

In [136]:
def home_screen_clicked(b):
  page.children = [create_home_screen()]

In [137]:
def disconnect_button_clicked(b):
  page.children = [create_login_screen()]
  login_dict.clear()

In [138]:
def create_menu():
  home_screen = widgets.Button(description="Home")
  home_screen.on_click(home_screen_clicked)

  parameters_screen = widgets.Button(description="Parameters", layout=widgets.Layout(disabled=True))
  parameters_screen.on_click(parameters_clicked)

  statistics_screen = widgets.Button(description="Statistics", layout=widgets.Layout(disabled=True))
  statistics_screen.on_click(statistics_clicked)

  json_upload_screen = widgets.Button(description="JSON Upload", layout=widgets.Layout(disabled=True))
  json_upload_screen.on_click(json_upload_clicked)

  index_search_screen = widgets.Button(description="Index Search", layout=widgets.Layout(disabled=True))
  index_search_screen.on_click(index_search_clicked)

  disconnect_button = widgets.Button(description="Disconnect", button_style="danger")
  disconnect_button.on_click(disconnect_button_clicked)

  menu_buttons_list.extend([parameters_screen, statistics_screen, json_upload_screen, index_search_screen])

  menu = widgets.HBox([home_screen, parameters_screen, statistics_screen, json_upload_screen, index_search_screen, disconnect_button])
  menu.add_class('menu')
  return menu

In [139]:
page = widgets.VBox(layout=widgets.Layout(width="100%", height="100%"))
page.add_class("page")
page_style = """
<style>
  .page {
    background-color: #ccd9ff;
    min-height: 875px;
  }
  .home-screen {
    margin: 8px auto;
  }
  .home-screen .buttons-vbox {
    margin-top: 32px;
  }
  .home-screen .widget-hbox {
    margin: 16px 0;
  }
  .home-screen .welcome-label {
    font-size: 40px;
    background: linear-gradient(to right, #f32170, #ff6b08, #cf23cf, #278a41);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    height: 52px;
    margin: 16px auto;
    padding-top: 4px;
  }
  .home-screen .widget-button {
    font-size: 32px;
    height: 300px;
    width: 300px;
    margin: 0 64px;
    background-color: #ccd9ff;
    color: #CFFF00;
    background-repeat: no-repeat;
    background-position: center;
    background-size: 200px;
    border-radius: 50%;
  }
  .home-screen .widget-button:hover {
    background-color: #809fff;
  }
  .home-screen .widget-button:active {
    background-color: #668cff;
  }
  .home-screen .ju {
    background-image: url("https://www.taniarascia.com/static/9d9dd7f6cf3b89757f9b4e17227ee5a5/92ab1/json.png");
  }
  .home-screen .is {
    background-image: url("https://images.pexels.com/photos/4494642/pexels-photo-4494642.jpeg?auto=compress&cs=tinysrgb&w=600");
  }
  .home-screen .ps {
    background-image: url("https://cdn.pixabay.com/photo/2017/05/09/10/03/equalizer-2297756_1280.png");
  }
  .home-screen .sts {
    background-image: url("https://cdn.sanity.io/images/tlr8oxjg/production/c277918894f2a79f6a029ffaecdcf3f22c7281b2-1456x816.png?w=3840&q=80&fit=clip&auto=format");
  }
  .menu .widget-button {
    background-color: #b3c6ff;
    color: black;
    margin: 4px 0 0 0.3em;
  }
  .menu .widget-button:hover {
    font-weight: bold;
  }
  .login-screen .widget-label {
    color: black;
  }
  .login-screen {
    border: 4px solid #002080;
    margin: 0 128px;
  }
  .param-menu .widget-label {
    color: black;
  }
  .statistics-screen .widget-label {
    color: black;
  }
  .statistics-output {
    color: black;
  }
  .statistics-output .widget-label {
    color: black;
  }
</style>
"""
page.children = [create_login_screen()]

#Main Program#

In [140]:
FBConn = connect_to_DB(DB_url)
if not index_is_posted:
  post_index_to_DB()

display(HTML(page_style))
display(page)